In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
master_df = pd.read_csv('master_data_table.csv')
feed_df = pd.read_csv('./Example Inputs/feed5.csv')

df = master_df.merge(feed_df, how='outer', on='entry').dropna()

df.head()

,Unnamed: 0,wals code,name,value,description_x,latitude,longitude,area,title,entry,description_y
268,9.0,agh,Aghem,3.0,Average,6.666667,10.000000,Phonology,Consonant Inventories,Consonant Inventories 3,Average
269,11.0,aik,Aikaná,3.0,Average,-12.666667,-60.666667,Phonology,Consonant Inventories,Consonant Inventories 3,Average
270,13.0,aiz,Aizi,3.0,Average,5.250000,-4.500000,Phonology,Consonant Inventories,Consonant Inventories 3,Average
271,14.0,akn,Akan,3.0,Average,6.500000,-1.250000,Phonology,Consonant Inventories,Consonant Inventories 3,Average
272,18.0,alw,Alawa,3.0,Average,-15.166667,134.250000,Phonology,Consonant Inventories,Consonant Inventories 3,Average


In [3]:
# # location = list(zip(df.latitude, df.longitude))
# df['latitude'] = latitude
df['entry'] = df['entry'].str.replace('/', ' or ')
df['entry'] = df['entry'].str.replace(' ', '_')

In [4]:
df

,Unnamed: 0,wals code,name,value,description_x,latitude,longitude,area,title,entry,description_y
268,9.0,agh,Aghem,3.0,Average,6.666667,10.000000,Phonology,Consonant Inventories,Consonant_Inventories_3,Average
269,11.0,aik,Aikaná,3.0,Average,-12.666667,-60.666667,Phonology,Consonant Inventories,Consonant_Inventories_3,Average
270,13.0,aiz,Aizi,3.0,Average,5.250000,-4.500000,Phonology,Consonant Inventories,Consonant_Inventories_3,Average
271,14.0,akn,Akan,3.0,Average,6.500000,-1.250000,Phonology,Consonant Inventories,Consonant_Inventories_3,Average
272,18.0,alw,Alawa,3.0,Average,-15.166667,134.250000,Phonology,Consonant Inventories,Consonant_Inventories_3,Average
...,...,...,...,...,...,...,...,...,...,...,...
75179,75335.0,yaq,Yaqui,8.0,No SONegV,27.500000,-110.250000,Word Order,SONegV Order,SONegV_Order_8,No SONegV
75180,75336.0,yar,Yareba,8.0,No SONegV,-9.500000,148.500000,Word Order,SONegV Order,SONegV_Order_8,No SONegV
75181,75340.0,yuw,Yuwaalaraay,8.0,No SONegV,-29.500000,148.000000,Word Order,SONegV Order,SONegV_Order_8,No SONegV
75182,75341.0,zay,Zayse,8.0,No SONegV,5.916667,37.333333,Word Order,SONegV Order,SONegV_Order_8,No SONegV


In [5]:
# entry = df['entry']
# dummies_entry = pd.get_dummies(entry)
# df['dummies_entry'] = dummies_entry
# df

In [6]:
X = df['entry']
y = df['latitude']

X.shape, y.shape

((8799,), (8799,))

In [7]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(X)
encoded_X = label_encoder.transform(X)

In [8]:
for label, original_class in zip(encoded_X, X):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Cons

------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label

Encoded Label: 15
------------
Original Class: Obligatory_Possessive_Inflection_2
Encoded Label: 15
------------
Original Class: Obligatory_Possessive_Inflection_2
Encoded Label: 15
------------
Original Class: Obligatory_Possessive_Inflection_2
Encoded Label: 15
------------
Original Class: Obligatory_Possessive_Inflection_2
Encoded Label: 15
------------
Original Class: Obligatory_Possessive_Inflection_2
Encoded Label: 15
------------
Original Class: Obligatory_Possessive_Inflection_2
Encoded Label: 15
------------
Original Class: Obligatory_Possessive_Inflection_2
Encoded Label: 15
------------
Original Class: Obligatory_Possessive_Inflection_2
Encoded Label: 15
------------
Original Class: Obligatory_Possessive_Inflection_2
Encoded Label: 15
------------
Original Class: Obligatory_Possessive_Inflection_2
Encoded Label: 15
------------
Original Class: Obligatory_Possessive_Inflection_2
Encoded Label: 15
------------
Original Class: Obligatory_Possessive_Inflection_2
Encoded Label: 1

------------
Original Class: Order_of_Subject,_Object_and_Verb_2
Encoded Label: 21
------------
Original Class: Order_of_Subject,_Object_and_Verb_2
Encoded Label: 21
------------
Original Class: Order_of_Subject,_Object_and_Verb_2
Encoded Label: 21
------------
Original Class: Order_of_Subject,_Object_and_Verb_2
Encoded Label: 21
------------
Original Class: Order_of_Subject,_Object_and_Verb_2
Encoded Label: 21
------------
Original Class: Order_of_Subject,_Object_and_Verb_2
Encoded Label: 21
------------
Original Class: Order_of_Subject,_Object_and_Verb_2
Encoded Label: 21
------------
Original Class: Order_of_Subject,_Object_and_Verb_2
Encoded Label: 21
------------
Original Class: Order_of_Subject,_Object_and_Verb_2
Encoded Label: 21
------------
Original Class: Order_of_Subject,_Object_and_Verb_2
Encoded Label: 21
------------
Original Class: Order_of_Subject,_Object_and_Verb_2
Encoded Label: 21
------------
Original Class: Order_of_Subject,_Object_and_Verb_2
Encoded Label: 21
----

Original Class: Order_of_Object_and_Verb_2
Encoded Label: 20
------------
Original Class: Order_of_Object_and_Verb_2
Encoded Label: 20
------------
Original Class: Order_of_Object_and_Verb_2
Encoded Label: 20
------------
Original Class: Order_of_Object_and_Verb_2
Encoded Label: 20
------------
Original Class: Order_of_Object_and_Verb_2
Encoded Label: 20
------------
Original Class: Order_of_Object_and_Verb_2
Encoded Label: 20
------------
Original Class: Order_of_Object_and_Verb_2
Encoded Label: 20
------------
Original Class: Order_of_Object_and_Verb_2
Encoded Label: 20
------------
Original Class: Order_of_Object_and_Verb_2
Encoded Label: 20
------------
Original Class: Order_of_Object_and_Verb_2
Encoded Label: 20
------------
Original Class: Order_of_Object_and_Verb_2
Encoded Label: 20
------------
Original Class: Order_of_Object_and_Verb_2
Encoded Label: 20
------------
Original Class: Order_of_Object_and_Verb_2
Encoded Label: 20
------------
Original Class: Order_of_Object_and_Ve

...

Encoded Label: 16
------------
Original Class: Order_of_Adjective_and_Noun_2
Encoded Label: 16
------------
Original Class: Order_of_Adjective_and_Noun_2
Encoded Label: 16
------------
Original Class: Order_of_Adjective_and_Noun_2
Encoded Label: 16
------------
Original Class: Order_of_Adjective_and_Noun_2
Encoded Label: 16
------------
Original Class: Order_of_Adjective_and_Noun_2
Encoded Label: 16
------------
Original Class: Order_of_Adjective_and_Noun_2
Encoded Label: 16
------------
Original Class: Order_of_Adjective_and_Noun_2
Encoded Label: 16
------------
Original Class: Order_of_Adjective_and_Noun_2
Encoded Label: 16
------------
Original Class: Order_of_Adjective_and_Noun_2
Encoded Label: 16
------------
Original Class: Order_of_Adjective_and_Noun_2
Encoded Label: 16
------------
Original Class: Order_of_Adjective_and_Noun_2
Encoded Label: 16
------------
Original Class: Order_of_Adjective_and_Noun_2
Encoded Label: 16
------------
Original Class: Order_of_Adjective_and_Noun_2

Encoded Label: 5
------------
Original Class: Ditransitive_Constructions_1
Encoded Label: 5
------------
Original Class: Ditransitive_Constructions_1
Encoded Label: 5
------------
Original Class: Ditransitive_Constructions_1
Encoded Label: 5
------------
Original Class: Ditransitive_Constructions_1
Encoded Label: 5
------------
Original Class: Ditransitive_Constructions_1
Encoded Label: 5
------------
Original Class: Ditransitive_Constructions_1
Encoded Label: 5
------------
Original Class: Ditransitive_Constructions_1
Encoded Label: 5
------------
Original Class: Ditransitive_Constructions_1
Encoded Label: 5
------------
Original Class: Ditransitive_Constructions_1
Encoded Label: 5
------------
Original Class: Ditransitive_Constructions_1
Encoded Label: 5
------------
Original Class: Ditransitive_Constructions_1
Encoded Label: 5
------------
Original Class: Ditransitive_Constructions_1
Encoded Label: 5
------------
Original Class: Ditransitive_Constructions_1
Encoded Label: 5
--------

------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_

Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negativ

Encoded Label: 28
------------
Original Class: SONegV_Order_8
Encoded Label: 28
------------
Original Class: SONegV_Order_8
Encoded Label: 28
------------
Original Class: SONegV_Order_8
Encoded Label: 28
------------
Original Class: SONegV_Order_8
Encoded Label: 28
------------
Original Class: SONegV_Order_8
Encoded Label: 28
------------
Original Class: SONegV_Order_8
Encoded Label: 28
------------
Original Class: SONegV_Order_8
Encoded Label: 28
------------
Original Class: SONegV_Order_8
Encoded Label: 28
------------
Original Class: SONegV_Order_8
Encoded Label: 28
------------
Original Class: SONegV_Order_8
Encoded Label: 28
------------
Original Class: SONegV_Order_8
Encoded Label: 28
------------
Original Class: SONegV_Order_8
Encoded Label: 28
------------
Original Class: SONegV_Order_8
Encoded Label: 28
------------
Original Class: SONegV_Order_8
Encoded Label: 28
------------
Original Class: SONegV_Order_8
Encoded Label: 28
------------
Original Class: SONegV_Order_8
Encoded 

In [9]:
one_hot_X = to_categorical(encoded_X)
# cols = [1,2,3,4,5,6,7,8,9,10]
# df = df[df.columns[cols]]
# df
one_hot_X[0:3,0:5]

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [10]:
# # Step 1: Label-encode data set
# label_encoder = LabelEncoder()
# label_encoder.fit(y)
# encoded_y = label_encoder.transform(y)

In [11]:
# create training and testing data
X_train, X_test, y_train, y_test = train_test_split(one_hot_X, y, random_state=1)

In [12]:
from sklearn.linear_model import LinearRegression
classifier = LinearRegression()
classifier

LinearRegression()

In [13]:
classifier.fit(one_hot_X, y)

LinearRegression()

In [14]:
from sklearn.metrics import mean_squared_error, r2_score

# Use our model to predict a value
predicted = classifier.predict(one_hot_X)

# Score the prediction with mse and r2
mse = mean_squared_error(y, predicted)
r2 = r2_score(y, predicted)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

Mean Squared Error (MSE): 472.214690665081
R-squared (R2 ): 0.047893758020811195


In [15]:
predicted_lat = classifier.predict(one_hot_X)

In [16]:
predicted_lat_avg = np.average(predicted_lat)
print(predicted_lat_avg)

10.959083


In [17]:
X = df['entry']
y = df['longitude']

X.shape, y.shape

((8799,), (8799,))

In [18]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(X)
encoded_X = label_encoder.transform(X)

In [19]:
for label, original_class in zip(encoded_X, X):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Consonant_Inventories_3
Encoded Label: 4
------------
Original Class: Cons

------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label

Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
------------
Original Class: Uvular_Consonants_1
Encoded Label: 34
--------

------------
Original Class: Gender_Distinctions_in_Independent_Personal_Pronouns_6
Encoded Label: 8
------------
Original Class: Gender_Distinctions_in_Independent_Personal_Pronouns_6
Encoded Label: 8
------------
Original Class: Gender_Distinctions_in_Independent_Personal_Pronouns_6
Encoded Label: 8
------------
Original Class: Gender_Distinctions_in_Independent_Personal_Pronouns_6
Encoded Label: 8
------------
Original Class: Gender_Distinctions_in_Independent_Personal_Pronouns_6
Encoded Label: 8
------------
Original Class: Gender_Distinctions_in_Independent_Personal_Pronouns_6
Encoded Label: 8
------------
Original Class: Gender_Distinctions_in_Independent_Personal_Pronouns_6
Encoded Label: 8
------------
Original Class: Gender_Distinctions_in_Independent_Personal_Pronouns_6
Encoded Label: 8
------------
Original Class: Gender_Distinctions_in_Independent_Personal_Pronouns_6
Encoded Label: 8
------------
Original Class: Gender_Distinctions_in_Independent_Personal_Pronouns_6
Encoded

Encoded Label: 9
------------
Original Class: Indefinite_Pronouns_1
Encoded Label: 9
------------
Original Class: Indefinite_Pronouns_1
Encoded Label: 9
------------
Original Class: Indefinite_Pronouns_1
Encoded Label: 9
------------
Original Class: Indefinite_Pronouns_1
Encoded Label: 9
------------
Original Class: Indefinite_Pronouns_1
Encoded Label: 9
------------
Original Class: Indefinite_Pronouns_1
Encoded Label: 9
------------
Original Class: Indefinite_Pronouns_1
Encoded Label: 9
------------
Original Class: Indefinite_Pronouns_1
Encoded Label: 9
------------
Original Class: Indefinite_Pronouns_1
Encoded Label: 9
------------
Original Class: Indefinite_Pronouns_1
Encoded Label: 9
------------
Original Class: Indefinite_Pronouns_1
Encoded Label: 9
------------
Original Class: Indefinite_Pronouns_1
Encoded Label: 9
------------
Original Class: Indefinite_Pronouns_1
Encoded Label: 9
------------
Original Class: Indefinite_Pronouns_1
Encoded Label: 9
------------
Original Class: In

...

Original Class: Subtypes_of_Asymmetric_Standard_Negation_5
Encoded Label: 29
------------
Original Class: Subtypes_of_Asymmetric_Standard_Negation_5
Encoded Label: 29
------------
Original Class: Subtypes_of_Asymmetric_Standard_Negation_5
Encoded Label: 29
------------
Original Class: Subtypes_of_Asymmetric_Standard_Negation_5
Encoded Label: 29
------------
Original Class: Subtypes_of_Asymmetric_Standard_Negation_5
Encoded Label: 29
------------
Original Class: Subtypes_of_Asymmetric_Standard_Negation_5
Encoded Label: 29
------------
Original Class: Subtypes_of_Asymmetric_Standard_Negation_5
Encoded Label: 29
------------
Original Class: Zero_Copula_for_Predicate_Nominals_1
Encoded Label: 35
------------
Original Class: Zero_Copula_for_Predicate_Nominals_1
Encoded Label: 35
------------
Original Class: Zero_Copula_for_Predicate_Nominals_1
Encoded Label: 35
------------
Original Class: Zero_Copula_for_Predicate_Nominals_1
Encoded Label: 35
------------
Original Class: Zero_Copula_for_Pr

Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encoded Label: 6
------------
Original Class: Finger_and_Hand_2
Encode

Original Class: M_in_Second_Person_Singular_2
Encoded Label: 11
------------
Original Class: M_in_Second_Person_Singular_2
Encoded Label: 11
------------
Original Class: M_in_Second_Person_Singular_2
Encoded Label: 11
------------
Original Class: M_in_Second_Person_Singular_2
Encoded Label: 11
------------
Original Class: M_in_Second_Person_Singular_2
Encoded Label: 11
------------
Original Class: M_in_Second_Person_Singular_2
Encoded Label: 11
------------
Original Class: M_in_Second_Person_Singular_2
Encoded Label: 11
------------
Original Class: M_in_Second_Person_Singular_2
Encoded Label: 11
------------
Original Class: M_in_Second_Person_Singular_2
Encoded Label: 11
------------
Original Class: M_in_Second_Person_Singular_2
Encoded Label: 11
------------
Original Class: M_in_Second_Person_Singular_2
Encoded Label: 11
------------
Original Class: M_in_Second_Person_Singular_2
Encoded Label: 11
------------
Original Class: M_in_Second_Person_Singular_2
Encoded Label: 11
------------

Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negative_Morphemes_1
Encoded Label: 27
------------
Original Class: Preverbal_Negativ

Original Class: Position_of_Negative_Word_With_Respect_to_Subject,_Object,_and_Verb_16
Encoded Label: 25
------------
Original Class: Position_of_Negative_Word_With_Respect_to_Subject,_Object,_and_Verb_16
Encoded Label: 25
------------
Original Class: Position_of_Negative_Word_With_Respect_to_Subject,_Object,_and_Verb_16
Encoded Label: 25
------------
Original Class: Position_of_Negative_Word_With_Respect_to_Subject,_Object,_and_Verb_16
Encoded Label: 25
------------
Original Class: Position_of_Negative_Word_With_Respect_to_Subject,_Object,_and_Verb_16
Encoded Label: 25
------------
Original Class: Position_of_Negative_Word_With_Respect_to_Subject,_Object,_and_Verb_16
Encoded Label: 25
------------
Original Class: Position_of_Negative_Word_With_Respect_to_Subject,_Object,_and_Verb_16
Encoded Label: 25
------------
Original Class: Position_of_Negative_Word_With_Respect_to_Subject,_Object,_and_Verb_16
Encoded Label: 25
------------
Original Class: Position_of_Negative_Word_With_Respect_t

In [20]:
one_hot_X = to_categorical(encoded_X)
one_hot_X[0:3,0:5]

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [21]:
# create training and testing data
X_train, X_test, y_train, y_test = train_test_split(one_hot_X, y, random_state=1)

In [22]:
classifier = LinearRegression()
classifier

classifier.fit(one_hot_X, y)

LinearRegression()

In [23]:
# Use our model to predict a value
predicted = classifier.predict(one_hot_X)

# Score the prediction with mse and r2
mse = mean_squared_error(y, predicted)
r2 = r2_score(y, predicted)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

predicted_lon = classifier.predict(one_hot_X)

predicted_lon_avg = np.average(predicted_lon)
print(predicted_lon_avg)

Mean Squared Error (MSE): 7838.1944125753525
R-squared (R2 ): 0.04784559358729412
31.797363


In [24]:
print(str(predicted_lat_avg) + ', ' + str(predicted_lon_avg))

10.959083, 31.797363


In [25]:
location_df = pd.DataFrame()
location_df['Latitude'] = predicted_lat
location_df['Longitude'] = predicted_lon
location_df

,Latitude,Longitude
0,16.21875,27.875
1,16.21875,27.875
2,16.21875,27.875
3,16.21875,27.875
4,16.21875,27.875
...,...,...
8794,11.75000,28.000
8795,11.75000,28.000
8796,11.75000,28.000
8797,11.75000,28.000
